Initial tests with deepseek code

In [6]:
from solana.rpc.api import Client
from solders.pubkey import Pubkey
from solders.rpc.responses import GetAccountInfoResp

def get_lp_mint_from_raydium_pool(client: Client, pool_address: Pubkey) -> Pubkey:
    """Extracts LP mint address from Raydium pool account data."""
    res = client.get_account_info(pool_address)
    if res.value is None:
        raise ValueError("Raydium pool account not found")
    data = res.value.data
    if len(data) < 368:  # Ensure data is long enough
        raise ValueError("Invalid Raydium pool account data")
    lp_mint_bytes = bytes(data[336:368])  # LP mint offset in Raydium pool
    return Pubkey.from_bytes(lp_mint_bytes)

def check_lp_token_status(client: Client, lp_mint: Pubkey) -> dict:
    """Checks LP token's supply and mint authority status."""
    res = client.get_account_info(lp_mint)
    if res.value is None:
        raise ValueError("LP mint account not found")
    data = res.value.data
    
    # Parse mint authority (bytes 0-33: COption<Pubkey>)
    mint_authority_tag = data[0]
    mint_authority = Pubkey.from_bytes(data[1:33]) if mint_authority_tag == 1 else None
    
    # Parse total supply (u64 at bytes 33-41)
    supply = int.from_bytes(data[33:41], 'little')
    
    return {
        "lp_burned": supply == 0,
        "mint_authority_revoked": mint_authority is None,
        "total_supply": supply
    }

def check_pool_ownership(client: Client, pool_address: Pubkey) -> bool:
    """Checks if pool is owned by Raydium's AMM program."""
    RAYDIUM_AMM_PROGRAM = Pubkey.from_string("675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8")
    res = client.get_account_info(pool_address)
    return res.value.owner == RAYDIUM_AMM_PROGRAM

def check_liquidity_locked(client: Client, lp_mint: Pubkey) -> bool:
    """Checks if largest LP token holder is a program account (potential lock)."""
    res = client.get_token_largest_accounts(lp_mint)
    if not res.value:
        return False
    largest_account = res.value[0].address
    account_info = client.get_account_info(largest_account)
    return account_info.value.owner.is_program if account_info.value else False

def check_solana_liquidity(pool_address: Pubkey) -> dict:
    client = Client("https://api.mainnet-beta.solana.com")
    
    try:
        lp_mint = get_lp_mint_from_raydium_pool(client, pool_address)
        token_status = check_lp_token_status(client, lp_mint)
        is_raydium = check_pool_ownership(client, pool_address)
        locked = check_liquidity_locked(client, lp_mint)
        
        return {
            "lp_burned": token_status["lp_burned"],
            "liquidity_locked": locked,
            "mint_authority_revoked": token_status["mint_authority_revoked"],
            "total_supply": token_status["total_supply"],
            "raydium_managed": is_raydium,
            "message": "Lock status checks if largest holder is a program"
        }
    except Exception as e:
        return {"error": str(e)}

# Example usage
if __name__ == "__main__":
    # Replace with actual Raydium pool address
    POOL_ADDRESS = Pubkey.from_string("DeAWLtRGAqCW7imV1jSC9NZkRKKGStFC1U7eDFBRxryR")
    result = check_solana_liquidity(POOL_ADDRESS)
    print(result)

# check_solana_liquidity(POOL_ADDRESS)

{'error': ''}


In [10]:
import requests

def find_pools_dexscreener(token_address: str):
    url = f"https://api.dexscreener.com/latest/dex/tokens/{token_address}"
    response = requests.get(url)
    return response.json().get("pairs", [])

# Get the token address
token_address = "H3bZqrnR9j3aYK3aNUEDynkcXHwbwKLbVgArAXAgpump"

# Call the function to get the pairs
pairs = find_pools_dexscreener(token_address)

# Initialize the variable for the Raydium pool address
YOUR_POOL_ADDRESS_HERE = None

# Loop through the pairs to find the Raydium pool
for pair in pairs:
    if pair.get('dexId') == 'raydium':
        YOUR_POOL_ADDRESS_HERE = pair.get('pairAddress')
        break  # Exit the loop once the Raydium pool is found

# Print the result to verify
print("YOUR_POOL_ADDRESS_HERE:", YOUR_POOL_ADDRESS_HERE)

YOUR_POOL_ADDRESS_HERE: 8bxMZZit3ZUc9m84TLfhn792PJwfUaCXn3eQeNjq6jyd


In [7]:
from solana.rpc.api import Client
from solders.pubkey import Pubkey
from solders.rpc.responses import GetAccountInfoResp
from solders.account import Account

# Initialize Solana client
solana_client = Client("https://api.mainnet-beta.solana.com")

# Known addresses
BURN_ADDRESS = Pubkey.from_string("11111111111111111111111111111111")
RAYDIUM_AMM_PROGRAM = Pubkey.from_string("675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8")
RAYDIUM_LOCKER = Pubkey.from_string("6WecgUmh3XJ7h56kwxjy1uTAcsz7qWQCjnd6Y2GZ5t8k")

def get_token_supply(mint_address: Pubkey) -> dict:
    """Get total supply and mint authority of a token"""
    try:
        resp = solana_client.get_account_info(mint_address)
        data = resp.value.data
        supply = int.from_bytes(data[36:44], 'little')
        mint_authority = Pubkey.from_bytes(data[0:32]) if int.from_bytes(data[32:36], 'little') else None
        return {"supply": supply, "mint_authority": mint_authority}
    except:
        return {"error": "Failed to fetch token info"}

def find_pools_dexscreener(token_address: str):
    url = f"https://api.dexscreener.com/latest/dex/tokens/{token_address}"
    response = requests.get(url)
    return response.json().get("pairs", [])

# Get the token address
token_address = "H3bZqrnR9j3aYK3aNUEDynkcXHwbwKLbVgArAXAgpump"

# Call the function to get the pairs
pairs = find_pools_dexscreener(token_address)

# Initialize the variable for the Raydium pool address
YOUR_POOL_ADDRESS_HERE = None

# Loop through the pairs to find the Raydium pool
for pair in pairs:
    if pair.get('dexId') == 'raydium':
        YOUR_POOL_ADDRESS_HERE = pair.get('pairAddress')
        break  # Exit the loop once the Raydium pool is found

def get_raydium_pool_info(pool_address: Pubkey) -> dict:
    """Extract liquidity pool information from Raydium pool account"""
    try:
        resp = solana_client.get_account_info(pool_address)
        data = resp.value.data
        
        # Offsets for Raydium pool structure
        token_a_amount = int.from_bytes(data[96:104], 'little')
        token_b_amount = int.from_bytes(data[200:208], 'little')
        lp_mint = Pubkey.from_bytes(data[336:368])
        
        return {
            "token_a": token_a_amount,
            "token_b": token_b_amount,
            "lp_mint": lp_mint
        }
    except:
        return {"error": "Invalid Raydium pool"}

def check_lp_status(lp_mint: Pubkey) -> dict:
    """Check LP token status (burned/locked)"""
    try:
        # Get LP token supply
        lp_info = get_token_supply(lp_mint)
        if lp_info["supply"] == 0:
            return {"status": "burned", "details": lp_info}
        
        # Check largest holder
        holders = solana_client.get_token_largest_accounts(lp_mint).value
        if holders:
            largest_holder = holders[0].address
            holder_info = solana_client.get_account_info(largest_holder)
            
            # Check if holder is a program
            is_program = holder_info.value.owner.is_program if holder_info.value else False
            is_lock_contract = largest_holder == RAYDIUM_LOCKER
            
            return {
                "status": "locked" if is_program else "held",
                "largest_holder": str(largest_holder),
                "is_lock_contract": is_lock_contract,
                "supply": lp_info["supply"]
            }
        return {"status": "unknown"}
    except:
        return {"error": "LP check failed"}

def analyze_token_safety(token_mint: Pubkey):
    """Main analysis function"""
    report = {}
    
    # 1. Get token supply info
    token_info = get_token_supply(token_mint)
    report["total_supply"] = token_info.get("supply", 0)
    report["mint_authority_revoked"] = token_info.get("mint_authority") is None
    
    # 2. Find liquidity pools (simplified - would need actual pool discovery)
    # In production: Use DeXscreener API or scan transaction history
    # This example assumes you already have pool addresses
    pool_address = Pubkey.from_string("YOUR_POOL_ADDRESS_HERE")
    pool_info = get_raydium_pool_info(pool_address)
    
    # 3. Calculate LP allocation percentage
    if "token_a" in pool_info:
        lp_allocation = (pool_info["token_a"] / report["total_supply"]) * 100
        report["lp_allocation"] = round(lp_allocation, 2)
        
        # 4. Check LP token status
        lp_mint = pool_info.get("lp_mint")
        if lp_mint:
            lp_status = check_lp_status(lp_mint)
            report["lp_status"] = lp_status
    
    # 5. Generate safety conclusion
    safety_score = 0
    if report.get("mint_authority_revoked"):
        safety_score += 1
    if report.get("lp_allocation", 0) < 40:
        safety_score += 1
    if report.get("lp_status", {}).get("status") == "burned":
        safety_score += 2
    elif report.get("lp_status", {}).get("status") == "locked":
        safety_score += 1
        
    report["safety_score"] = safety_score
    report["verdict"] = "Safe" if safety_score >= 3 else "Caution" if safety_score >=2 else "Risky"
    
    return report

# Example usage
if __name__ == "__main__":
    TOKEN_ADDRESS = Pubkey.from_string("H3bZqrnR9j3aYK3aNUEDynkcXHwbwKLbVgArAXAgpump")
    result = analyze_token_safety(TOKEN_ADDRESS)
    print(result)

ValueError: Invalid Base58 string

In [8]:
from solana.rpc.api import Client
from solders.pubkey import Pubkey
import requests

# Initialize Solana client
solana_client = Client("https://api.mainnet-beta.solana.com")

# Known addresses
BURN_ADDRESS = Pubkey.from_string("11111111111111111111111111111111")
RAYDIUM_LOCKER = Pubkey.from_string("6WecgUmh3XJ7h56kwxjy1uTAcsz7qWQCjnd6Y2GZ5t8k")

def find_raydium_pool(token_address: str) -> str:
    """Find Raydium pool address using DexScreener API"""
    url = f"https://api.dexscreener.com/latest/dex/tokens/{token_address}"
    try:
        response = requests.get(url, timeout=10)
        pairs = response.json().get("pairs", [])
        for pair in pairs:
            if pair.get('dexId') == 'raydium':
                return pair.get('pairAddress')
        return None
    except Exception as e:
        print(f"DexScreener API error: {str(e)}")
        return None

def get_token_supply(mint_address: Pubkey) -> int:
    """Get total supply of a token"""
    try:
        resp = solana_client.get_account_info(mint_address)
        data = resp.value.data
        return int.from_bytes(data[36:44], 'little')
    except:
        return None

def get_raydium_pool_info(pool_address: Pubkey) -> dict:
    """Extract liquidity pool information from Raydium pool account"""
    try:
        resp = solana_client.get_account_info(pool_address)
        data = resp.value.data
        
        # Offsets for Raydium pool structure
        token_a_amount = int.from_bytes(data[96:104], 'little')
        token_b_amount = int.from_bytes(data[200:208], 'little')
        lp_mint = Pubkey.from_bytes(data[336:368])
        
        return {
            "token_a": token_a_amount,
            "token_b": token_b_amount,
            "lp_mint": lp_mint
        }
    except:
        return {"error": "Invalid Raydium pool"}

def check_lp_status(lp_mint: Pubkey) -> dict:
    """Check LP token status (burned/locked)"""
    try:
        # Get LP token supply
        lp_supply = get_token_supply(lp_mint)
        if lp_supply is None:
            return {"error": "Failed to get LP token supply"}
        
        if lp_supply == 0:
            return {"status": "burned", "supply": 0}
        
        # Check largest holder
        try:
            holders_response = solana_client.get_token_largest_accounts(lp_mint)
            holders = holders_response.value if holders_response else None
            
            if not holders or len(holders) == 0:
                return {"status": "unknown", "reason": "No holders found"}
                
            largest_holder = holders[0].address
            holder_info = solana_client.get_account_info(largest_holder)
            
            # Check if holder is a program
            is_program = holder_info.value.owner.is_program if holder_info.value else False
            is_lock_contract = largest_holder == RAYDIUM_LOCKER
            
            return {
                "status": "locked" if is_program else "held",
                "largest_holder": str(largest_holder),
                "is_lock_contract": is_lock_contract,
                "supply": lp_supply
            }
        except Exception as e:
            return {"error": f"LP holders check failed: {str(e)}"}
    except Exception as e:
        return {"error": f"LP check failed: {str(e)}"}

def analyze_token_liquidity(token_address: str):
    """Main analysis function"""
    report = {}
    
    # 1. Find Raydium pool
    pool_address = find_raydium_pool(token_address)
    if not pool_address:
        return {"error": "No Raydium pool found"}
    
    report["pool_address"] = pool_address
    
    # 2. Get pool info
    pool_info = get_raydium_pool_info(Pubkey.from_string(pool_address))
    if "error" in pool_info:
        return pool_info
    
    # 3. Get token supply and calculate allocation
    token_supply = get_token_supply(Pubkey.from_string(token_address))
    if not token_supply:
        return {"error": "Failed to get token supply"}
    
    report["total_supply"] = token_supply
    report["lp_allocation"] = (pool_info["token_a"] / token_supply) * 100
    
    # 4. Check LP token status
    lp_mint = pool_info.get("lp_mint")
    if lp_mint:
        lp_status = check_lp_status(lp_mint)
        report["lp_status"] = lp_status
    
    # 5. Generate safety conclusion
    safety_score = 0
    if report.get("lp_allocation", 0) < 40:
        safety_score += 1
    if report.get("lp_status", {}).get("status") == "burned":
        safety_score += 2
    elif report.get("lp_status", {}).get("status") == "locked":
        safety_score += 1
        
    report["safety_score"] = safety_score
    report["verdict"] = "Safe" if safety_score >= 3 else "Caution" if safety_score >=2 else "Risky"
    
    return report

# Example usage
if __name__ == "__main__":
    TOKEN_ADDRESS = "H3bZqrnR9j3aYK3aNUEDynkcXHwbwKLbVgArAXAgpump"  # Replace with your token
    result = analyze_token_liquidity(TOKEN_ADDRESS)
    print(result)

{'pool_address': '8bxMZZit3ZUc9m84TLfhn792PJwfUaCXn3eQeNjq6jyd', 'total_supply': 999455228082232, 'lp_allocation': 1.000545068855974e-06, 'lp_status': {'error': 'LP holders check failed: '}, 'safety_score': 1, 'verdict': 'Risky'}


In [9]:
from solana.rpc.api import Client
from solders.pubkey import Pubkey
import requests

# Initialize Solana client
solana_client = Client("https://api.mainnet-beta.solana.com")

# Known addresses
BURN_ADDRESS = Pubkey.from_string("11111111111111111111111111111111")
RAYDIUM_LOCKER = Pubkey.from_string("6WecgUmh3XJ7h56kwxjy1uTAcsz7qWQCjnd6Y2GZ5t8k")

def find_raydium_pool(token_address: str) -> str:
    """Find Raydium pool address using DexScreener API"""
    url = f"https://api.dexscreener.com/latest/dex/tokens/{token_address}"
    try:
        response = requests.get(url, timeout=10)
        pairs = response.json().get("pairs", [])
        for pair in pairs:
            if pair.get('dexId') == 'raydium' and pair.get('quoteToken', {}).get('symbol') == 'USDC':
                return pair.get('pairAddress')
        return None
    except Exception as e:
        print(f"DexScreener API error: {str(e)}")
        return None

def get_token_supply(mint_address: Pubkey) -> int:
    """Get total supply of a token"""
    try:
        resp = solana_client.get_account_info(mint_address)
        if resp.value is None:
            return None
        data = resp.value.data
        return int.from_bytes(data[36:44], 'little')
    except Exception as e:
        print(f"Token supply error: {str(e)}")
        return None

def get_raydium_pool_info(pool_address: Pubkey) -> dict:
    """Improved Raydium pool data parser"""
    try:
        resp = solana_client.get_account_info(pool_address)
        if resp.value is None:
            return {"error": "Pool account not found"}
            
        data = resp.value.data
        if len(data) < 368:
            return {"error": "Invalid pool data length"}
        
        # Updated offsets based on Raydium pool structure
        token_a_amount = int.from_bytes(data[144:152], 'little')  # Base token amount
        token_b_amount = int.from_bytes(data[216:224], 'little')  # Quote token amount
        lp_mint = Pubkey.from_bytes(data[336:368])               # LP mint address
        
        return {
            "token_a": token_a_amount,
            "token_b": token_b_amount,
            "lp_mint": lp_mint
        }
    except Exception as e:
        print(f"Pool parse error: {str(e)}")
        return {"error": f"Failed to parse pool data: {str(e)}"}

def check_lp_status(lp_mint: Pubkey) -> dict:
    """Enhanced LP status checker"""
    try:
        # Get LP token supply
        lp_supply = get_token_supply(lp_mint)
        if lp_supply is None:
            return {"error": "Failed to get LP supply"}
            
        if lp_supply == 0:
            return {"status": "burned", "supply": 0}
        
        # Check largest holder
        holders = solana_client.get_token_largest_accounts(lp_mint)
        if holders.value is None or len(holders.value) == 0:
            return {"status": "unknown"}
            
        largest_holder = holders.value[0].address
        holder_info = solana_client.get_account_info(largest_holder)
        
        if holder_info.value is None:
            return {"status": "unknown"}
            
        # Check if holder is a program
        is_program = holder_info.value.owner.is_program
        is_lock_contract = largest_holder == RAYDIUM_LOCKER
        
        return {
            "status": "locked" if is_program else "held",
            "largest_holder": str(largest_holder),
            "is_lock_contract": is_lock_contract,
            "supply": lp_supply
        }
    except Exception as e:
        print(f"LP check error: {str(e)}")
        return {"error": f"LP check failed: {str(e)}"}

def analyze_token_liquidity(token_address: str):
    """Improved analysis with better error handling"""
    report = {}
    
    # 1. Find Raydium pool
    pool_address = find_raydium_pool(token_address)
    if not pool_address:
        return {"error": "No Raydium pool found"}
    report["pool_address"] = pool_address
    
    # 2. Get pool info
    pool_info = get_raydium_pool_info(Pubkey.from_string(pool_address))
    if "error" in pool_info:
        return pool_info
    
    # 3. Get token supply
    token_supply = get_token_supply(Pubkey.from_string(token_address))
    if not token_supply:
        return {"error": "Failed to get token supply"}
    report["total_supply"] = token_supply
    
    # 4. Calculate LP allocation percentage
    try:
        lp_allocation = (pool_info["token_a"] / token_supply) * 100
        report["lp_allocation"] = round(lp_allocation, 4)
    except KeyError:
        return {"error": "Missing token amount in pool data"}
    
    # 5. Check LP token status
    if "lp_mint" not in pool_info:
        return {"error": "Missing LP mint in pool data"}
        
    lp_status = check_lp_status(pool_info["lp_mint"])
    report["lp_status"] = lp_status
    
    # 6. Generate safety conclusion
    safety_score = 0
    
    # LP allocation scoring
    if report["lp_allocation"] < 40:
        safety_score += 1
    elif report["lp_allocation"] > 70:
        safety_score -= 1
        
    # LP status scoring
    if lp_status.get("status") == "burned":
        safety_score += 2
    elif lp_status.get("status") == "locked":
        safety_score += 1
    elif lp_status.get("status") == "held":
        safety_score -= 1
        
    # Final verdict
    report["safety_score"] = safety_score
    if safety_score >= 3:
        report["verdict"] = "Safe"
    elif safety_score >= 1:
        report["verdict"] = "Caution"
    else:
        report["verdict"] = "Risky"
    
    return report

# Example usage
if __name__ == "__main__":
    TOKEN_ADDRESS = "H3bZqrnR9j3aYK3aNUEDynkcXHwbwKLbVgArAXAgpump"  # Replace with your token
    result = analyze_token_liquidity(TOKEN_ADDRESS)
    print(result)

{'error': 'No Raydium pool found'}
